![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FApplied+ML%2FSolution+Prototypes%2Fdocument-processing%2F7-agents%2Fdocument_agent%2Fapps&file=app_testing.ipynb)
<!--- header table --->
<table align="left">
<tr>
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20ML/Solution%20Prototypes/document-processing/7-agents/document_agent/apps/app_testing.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20ML/Solution%20Prototypes/document-processing/7-agents/document_agent/apps/app_testing.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FApplied%2520ML%2FSolution%2520Prototypes%2Fdocument-processing%2F7-agents%2Fdocument_agent%2Fapps%2Fapp_testing.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20ML/Solution%20Prototypes/document-processing/7-agents/document_agent/apps/app_testing.ipynb">
      <img width="32px" src="https://www.gstatic.com/images/branding/gcpiconscolors/bigquery/v1/32px.svg" alt="BigQuery logo">
      <br>Open in<br>BigQuery Studio
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Applied%20ML/Solution%20Prototypes/document-processing/7-agents/document_agent/apps/app_testing.ipynb">
      <img width="32px" src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</tr>
<tr>
  <td colspan="5" style="text-align: right">
    <b>Share This On: </b> 
    <a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/statmike/vertex-ai-mlops/blob/main/Applied%20ML/Solution%20Prototypes/document-processing/7-agents/document_agent/apps/app_testing.ipynb"><img src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="Linkedin Logo" width="20px"></a> 
    <a href="https://reddit.com/submit?url=https%3A//github.com/statmike/vertex-ai-mlops/blob/main/Applied%20ML/Solution%20Prototypes/document-processing/7-agents/document_agent/apps/app_testing.ipynb"><img src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit Logo" width="20px"></a> 
    <a href="https://bsky.app/intent/compose?text=https%3A//github.com/statmike/vertex-ai-mlops/blob/main/Applied%20ML/Solution%20Prototypes/document-processing/7-agents/document_agent/apps/app_testing.ipynb"><img src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="BlueSky Logo" width="20px"></a> 
    <a href="https://twitter.com/intent/tweet?url=https%3A//github.com/statmike/vertex-ai-mlops/blob/main/Applied%20ML/Solution%20Prototypes/document-processing/7-agents/document_agent/apps/app_testing.ipynb"><img src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X (Twitter) Logo" width="20px"></a> 
  </td>
</tr>
<tr>
  <td colspan="5" style="text-align: right">
    <b>Connect With Author On: </b> 
    <a href="https://www.linkedin.com/in/statmike"><img src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="Linkedin Logo" width="20px"></a>
    <a href="https://www.github.com/statmike"><img src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub Logo" width="20px"></a> 
    <a href="https://www.youtube.com/@statmike-channel"><img src="https://upload.wikimedia.org/wikipedia/commons/f/fd/YouTube_full-color_icon_%282024%29.svg" alt="YouTube Logo" width="20px"></a>
    <a href="https://bsky.app/profile/statmike.bsky.social"><img src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="BlueSky Logo" width="20px"></a> 
    <a href="https://x.com/statmike"><img src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X (Twitter) Logo" width="20px"></a>
  </td>
</tr>
</table>

# REST Interact With `adk web`

Make sure you have started `adk web` locally.

In [1]:
!which python

/home/statmike/repos/vertex-ai-mlops/Applied ML/Solution Prototypes/document-processing/7-agents/.venv/bin/python


In [18]:
import requests
import json

In [4]:
ADK_SESSION_CREATE_URL = "http://localhost:8000/apps/document_agent/users/user/sessions"
ADK_RUN_SSE_URL = "http://localhost:8000/run_sse"
ADK_RUN_URL = "http://localhost:8000/run"

In [5]:
ADK_SESSION_CREATE_URL, ADK_RUN_SSE_URL, ADK_RUN_URL

('http://localhost:8000/apps/document_agent/users/user/sessions',
 'http://localhost:8000/run_sse',
 'http://localhost:8000/run')

## Get A Session

In [6]:
# override for testing
ADK_SESSION_CREATE_URL = 'http://localhost:8000/apps'

In [7]:
agent_name = 'document_agent'
user_id = 'u_123'
session_id = 's_123'

In [8]:
ADK_SESSION_URL = f"{ADK_SESSION_CREATE_URL}/{agent_name}/users/{user_id}/sessions/{session_id}"
ADK_SESSION_URL

'http://localhost:8000/apps/document_agent/users/u_123/sessions/s_123'

In [9]:
session_payload = {}
response = requests.post(ADK_SESSION_URL, json=session_payload, timeout=30)
response.raise_for_status()  # Will raise an exception for HTTP errors
response_data = response.json()
response_data

{'id': 's_123',
 'appName': 'document_agent',
 'userId': 'u_123',
 'state': {},
 'events': [],
 'lastUpdateTime': 1749657508.9713526}

In [10]:
SERVER_SESSION_ID = response_data['id']
SERVER_SESSION_ID

's_123'

## Interact With Session

In [11]:
message_parts = []
init_message = {'text': 'Hello'}
message_parts.append(init_message)

In [12]:
payload = {
    "appName": agent_name,
    "userId": user_id, 
    "sessionId": SERVER_SESSION_ID, 
    "newMessage": {
        "role": "user",
        "parts": message_parts
    },
    "streaming": False 
}

In [13]:
response = requests.post(ADK_RUN_URL, json=payload, timeout=180)
response.raise_for_status()
response_data = response.json()
response_data

[{'content': {'parts': [{'text': 'Hello! To start, please upload a PDF or PNG file, or provide the GCS URI (bucket and path) of the document you would like to process.\n'}],
   'role': 'model'},
  'usageMetadata': {'candidatesTokenCount': 34,
   'candidatesTokensDetails': [{'modality': 'TEXT', 'tokenCount': 34}],
   'promptTokenCount': 3470,
   'promptTokensDetails': [{'modality': 'TEXT', 'tokenCount': 3470}],
   'totalTokenCount': 3504,
   'trafficType': 'ON_DEMAND'},
  'invocationId': 'e-ba26d420-7883-4736-93be-9156a93bd9f2',
  'author': 'document_agent',
  'actions': {'stateDelta': {},
   'artifactDelta': {},
   'requestedAuthConfigs': {}},
  'id': 'xp2iFSq9',
  'timestamp': 1749657533.962014}]

In [14]:
def parse_response(response_data):
    final_text = "Error: No final response from assistant found."
    if isinstance(response_data, list):
        for message in reversed(response_data):
            content = message.get('content', {})
            if isinstance(content, dict) and content.get("role") == "model":
                parts = content.get('parts', [])
                if parts and isinstance(parts, list):
                    first_part = parts[0]
                    if isinstance(first_part, dict):
                        text_part = first_part.get("text")
                        if text_part:
                            final_text = text_part
                        else:
                            final_text = f"Agent returned a structured response: `{json.dumps(first_part)}`"
                    else:
                        final_text = f"Agent returned an unknown part: `{str(first_part)}`"
                    break
    return final_text

parse_response(response_data)

'Hello! To start, please upload a PDF or PNG file, or provide the GCS URI (bucket and path) of the document you would like to process.\n'

## Add To Interaction

In [78]:
message_parts = []
init_message = {'text': 'use this file: gs://statmike-mlops-349915/applied-ml-solution-prototypes/document-processing/vendor_5/fake_invoices/vendor_5_invoice_10.pdf'}
message_parts.append(init_message)

payload = {
    "appName": agent_name,
    "userId": user_id, 
    "sessionId": SERVER_SESSION_ID, 
    "newMessage": {
        "role": "user",
        "parts": message_parts
    },
    "streaming": False 
}

In [79]:
response = requests.post(ADK_RUN_SSE_URL, json=payload, timeout=180, stream = True)
response.raise_for_status()

In [81]:
for line in response.iter_lines(decode_unicode = True):
    if line:
        event = json.loads(line.split(':', 1)[1])
        if type(event) == list:
            for message in event:
                final_text = message.get("content", {}).get("parts", [{}])[0].get("text")
        else:
            final_text = event.get('content').get('parts')[0].get('text')

In [82]:
final_text

'I have already loaded the file located at `gs://statmike-mlops-349915/applied-ml-solution-prototypes/document-processing/vendor_5/fake_invoices/vendor_5_invoice_10.pdf`. Its artifact key is `gcsfile_statmike-mlops-349915_applied-ml-solution-prototypes_document-processing_vendor_5_fake_invoices_vendor_5_invoice_10.pdf`.\n\nPlease tell me what you would like to do with the loaded file. I can perform extraction, classification, or comparison.\n'

In [83]:
line

'data: {"content":{"parts":[{"text":"I have loaded the file located at `gs://statmike-mlops-349915/applied-ml-solution-prototypes/document-processing/vendor_5/fake_invoices/vendor_5_invoice_10.pdf`. The artifact key is `gcsfile_statmike-mlops-349915_applied-ml-solution-prototypes_document-processing_vendor_5_fake_invoices_vendor_5_invoice_10.pdf`.\\n\\nTo proceed, please tell me what you would like to do with the loaded document. I can perform one of the following actions:\\n\\n*   **Extraction:** Extract text and structure from the document.\\n*   **Classification:** Classify the document based on its content.\\n*   **Comparison:** Compare the document to a vendor template (requires classification first).\\n\\nPlease let me know your choice.\\n"}],"role":"model"},"usageMetadata":{"candidatesTokenCount":191,"candidatesTokensDetails":[{"modality":"TEXT","tokenCount":191}],"promptTokenCount":6099,"promptTokensDetails":[{"modality":"TEXT","tokenCount":6099}],"totalTokenCount":6290,"traffi

In [84]:
len(event)

7

## Fetch A Session

In [16]:
session_payload = {}
response = requests.get(ADK_SESSION_URL, json=session_payload, timeout=30)
response.raise_for_status()  # Will raise an exception for HTTP errors
response_data = response.json()
response_data

{'id': 's_123',
 'appName': 'document_agent',
 'userId': 'u_123',
 'state': {},
 'events': [{'content': {'parts': [{'text': 'Hello'}], 'role': 'user'},
   'invocationId': 'e-ac430462-e524-452a-9d5c-a50cd11b0b78',
   'author': 'user',
   'actions': {'stateDelta': {},
    'artifactDelta': {},
    'requestedAuthConfigs': {}},
   'id': 'ogFZi3nX',
   'timestamp': 1749237747.117274},
  {'content': {'parts': [{'text': 'Hello! To get started, please upload a PDF or PNG file, or provide the GCS URI (bucket and path) for the document you would like me to process.\n'}],
    'role': 'model'},
   'usageMetadata': {'candidatesTokenCount': 36,
    'candidatesTokensDetails': [{'modality': 'TEXT', 'tokenCount': 36}],
    'promptTokenCount': 3470,
    'promptTokensDetails': [{'modality': 'TEXT', 'tokenCount': 3470}],
    'totalTokenCount': 3506,
    'trafficType': 'ON_DEMAND'},
   'invocationId': 'e-ac430462-e524-452a-9d5c-a50cd11b0b78',
   'author': 'document_agent',
   'actions': {'stateDelta': {},
 

---
### NOTES

- [ ] If artifact_delta then can i retrieve the artifact locally?
- [ ] How can i send a file as inline data - see app implementation for example
- [X] can i retrieve the entire session from the URL?